In [7]:
# # images 폴더 초기화
# import os
# import shutil

# delete = ['images']

# for folder_path in delete:
#   for filename in os.listdir(folder_path):
#       file_path = os.path.join(folder_path, filename)
#       if os.path.isfile(file_path) or os.path.islink(file_path):
#           os.unlink(file_path)
#       elif os.path.isdir(file_path):
#           shutil.rmtree(file_path)

# Needs to change to google cloud storage

In [9]:
# video 폴더 읽기
import os

video_folder_path = 'video'

mp4_files = [file for file in os.listdir(video_folder_path) if file.endswith('.mp4')]

if len(mp4_files)>1:
  ValueError("video 폴더 안에 복수의 파일이 존재함: ", mp4_files)

mp4_files

['12345_2024-01-01_N33_E124.mp4']

In [10]:
# 메타 데이터 지정
video_name = mp4_files[0]
arr = video_name.split('_')

user_id = arr[0]
date = arr[1]
lat = arr[2]
lon = (arr[3]).replace('.mp4', '')

user_id, date, lat, lon

('12345', '2024-01-01', 'N33', 'E124')

In [11]:
import cv2

img_folder_path = 'images'
video_path = os.path.join(video_folder_path, video_name)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise Exception("Could not open the video file.")

frame_rate = cap.get(cv2.CAP_PROP_FPS)
interval = 5 # 인터벌 조정 가능. 실제 다이빙 영상에서는 10초 이상 권장

frame_count = 0
success, image = cap.read()
while success:
    if frame_count % (frame_rate * interval) == 0:
        cv2.imwrite(os.path.join(img_folder_path, f'{video_name}_{frame_count}.jpg'), image)

    success, image = cap.read()
    frame_count += 1

cap.release()


In [19]:
from ultralytics import YOLO
import os

model = YOLO('model.pt')
image_list = os.listdir('images')
image_list = [f"images/{i}" for i in image_list]
results = model(image_list)  # list of 2 Results objects




0: 448x800 (no detections), 1: 448x800 (no detections), 2: 448x800 1 Cyprinus carpio haematopterus, 3: 448x800 (no detections), 4: 448x800 1 Thalassoma bifasciatum, 5: 448x800 1 Naso lituratus, 1 Scarus rubroviolaceus, 6: 448x800 1 Sphyraena barracuda, 7: 448x800 2 Scarus rubroviolaceuss, 1 Sphyraena barracuda, 8: 448x800 1 Sphyraena barracuda, 9: 448x800 1 Sphyraena barracuda, 10: 448x800 1 Sphyraena barracuda, 11: 448x800 1 Megalops atlanticus, 2 Sphyraena barracudas, 12: 448x800 6 Megalops atlanticuss, 10442.4ms
Speed: 13.5ms preprocess, 803.3ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 800)


[]
[]
[
  {
    "name": "Cyprinus carpio haematopterus",
    "class": 13,
    "confidence": 0.50639808177948,
    "box": {
      "x1": 1159.2138671875,
      "y1": 949.9711303710938,
      "x2": 1361.1572265625,
      "y2": 1011.63232421875
    }
  }
]
[]
[
  {
    "name": "Thalassoma bifasciatum",
    "class": 51,
    "confidence": 0.3029719889163971,
    "box": {
      "x1": 1400.2266845703125,
      "y1": 286.052490234375,
      "x2": 1455.532470703125,
      "y2": 387.6289978027344
    }
  }
]
[
  {
    "name": "Scarus rubroviolaceus",
    "class": 47,
    "confidence": 0.6203732490539551,
    "box": {
      "x1": 0.53984534740448,
      "y1": 566.1338500976562,
      "x2": 267.2662353515625,
      "y2": 891.412841796875
    }
  },
  {
    "name": "Naso lituratus",
    "class": 36,
    "confidence": 0.5008959770202637,
    "box": {
      "x1": 1538.177978515625,
      "y1": 628.341552734375,
      "x2": 1711.5108642578125,
      "y2": 788.7944946289062
    }
  }
]
[
  {
    "name":

In [24]:
from PIL import Image
import json

# 결과 처리
json_data = []

for i, r in enumerate(results):
    img_path = image_list[i]
    image = Image.open(img_path)
    detections = json.loads(r.tojson())
    for detection in detections:
        if detection:

            # 바운딩 박스 추출 및 저장
            box = detection['box']
            cropped_image = image.crop((box['x1'], box['y1'], box['x2'], box['y2']))
            save_path = f"results/images/detected_{i}.png"
            cropped_image.save(save_path)

            # JSON 데이터 생성
            json_data.append({
                'user_id': user_id,
                'date': date,
                'lat': lat,
                'lon': lon,
                'species': detection['name'],
                'img_path': save_path
            })

# JSON 파일 저장
with open('results/result.json', 'w') as file:
    json.dump(json_data, file, indent = 4)